In [ ]:
!pip3 install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install jiwer
!pip3 install ds-ctcdecoder
!pip3 install levenshtein


In [ ]:
# Pulling datasets in
%ntbl pull datasets "BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon/Train.csv"

%ntbl pull datasets "BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon/Train.zip"

!unzip "../datasets/BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon/Train.zip" -d /tmp/data


In [ ]:
# File paths
MANIFEST_FILE_TRAIN = '../datasets/BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon/Train.csv'
AUDIO_DIR_TRAIN = '/tmp/data/Train/'
SAVED_MODEL_PATH = '/models/ASR50_beam_model.pt'

In [147]:
# import torch and os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torchaudio.datasets import SPEECHCOMMANDS
from torch.utils.data import DataLoader
import os

MANIFEST_FILE_TRAIN = 'src/dataset/trainshort.csv'
AUDIO_DIR_TRAIN = 'src/dataset/trainshort/trainshort/'
SAVED_MODEL_PATH = 'models/ASR50_beam_model.pt'

In [148]:
str(torchaudio.get_audio_backend())

'soundfile'

In [149]:
# Custom Speech Dataset class to load the dataset
import os
import pandas as pd
from typing import Tuple
import torch
import torchaudio


class CustomSpeechDataset(torch.utils.data.Dataset):
    
    """
    Custom torch dataset class to load the dataset 
    """
    
    def __init__(self, manifest_file: str, audio_dir: str, is_test_set: bool=False) -> None:

        """
        manifest_file: the csv file that contains the filename of the audio, and also the annotation if is_test_set is set to False
        audio_dir: the root directory of the audio datasets
        is_test_set: the flag variable to switch between loading of the train and the test set. Train set loads the annotation whereas test set does not
        """

        self.audio_dir = audio_dir
        self.is_test_set = is_test_set

        self.manifest = pd.read_csv(manifest_file)

        
    def __len__(self) -> int:
        
        """
        To get the number of loaded audio files in the dataset
        """

        return len(self.manifest)
    
    
    def __getitem__(self, index: int) -> Tuple[str, torch.Tensor]:

        """
        To get the values required to do the training
        """

        if torch.is_tensor(index):
            index.tolist()
            
        audio_path = self._get_audio_path(index)
        signal, sr = torchaudio.load(audio_path)
        
        if not self.is_test_set:
            annotation = self._get_annotation(index)
            return audio_path, signal, annotation
        
        return audio_path, signal
    
    
    def _get_audio_path(self, index: int) -> str:

        """
        Helper function to retrieve the audio path from the csv manifest file
        """
        
        path = os.path.join(self.audio_dir, self.manifest.iloc[index]['path'])

        return path
    
    
    def _get_annotation(self, index: int) -> str:

        """
        Helper function to retrieve the annotation from the csv manifest file
        """

        return self.manifest.iloc[index]['annotation']

In [150]:
# Create custom dataset
dataset = CustomSpeechDataset(
        manifest_file=MANIFEST_FILE_TRAIN, 
        audio_dir=AUDIO_DIR_TRAIN, 
        is_test_set=False
    )
# train_dev_split
train_proportion = int(0.8 * len(dataset))
dataset_train = list(dataset)[:train_proportion]
dataset_dev = list(dataset)[train_proportion:]

In [151]:
# Transforms text by encoding the characters and decoding the integers corresponding to the characters

from typing import List


class TextTransform:

    """
    Map characters to integers and vice versa (encoding/decoding)
    """
    
    def __init__(self) -> None:

        char_map_str = """
            <SPACE> 0
            A 1
            B 2
            C 3
            D 4
            E 5
            F 6
            G 7
            H 8
            I 9
            J 10
            K 11
            L 12
            M 13
            N 14
            O 15
            P 16
            Q 17
            R 18
            S 19
            T 20
            U 21
            V 22
            W 23
            X 24
            Y 25
            Z 26
        """
        
        self.char_map = {}
        self.index_map = {}
        
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch

        self.index_map[0] = ' '


    def get_char_len(self) -> int:

        """
        Gets the number of characters that are being encoded and decoded in the prediction
        Returns:
        --------
            the number of characters defined in the __init__ char_map_str
        """

        return len(self.char_map)
    

    def get_char_list(self) -> List[str]:

        """
        Gets the list of characters that are being encoded and decoded in the prediction
        
        Returns:
        -------
            a list of characters defined in the __init__ char_map_str
        """

        return list(self.index_map.values())
    

    def text_to_int(self, text: str) -> List[int]:

        """
        Use a character map and convert text to an integer sequence 
        Returns:
        -------
            a list of the text encoded to an integer sequence 
        """
        
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)

        return int_sequence
    

    def int_to_text(self, labels) -> str:

        """
        Use a character map and convert integer labels to an text sequence 
        
        Returns:
        -------
            the decoded transcription
        """
        
        string = []
        for i in labels:
            string.append(self.index_map[i])

        return ''.join(string).replace('<SPACE>', ' ')

In [152]:

# Data preprocessing and transformation of the audio files into melspectrogram

import torch
import torchaudio
import random

class DataProcessor:

    """
    Transforms the audio waveform tensors into a melspectrogram
    """

    def __init__(self) -> None:
        pass
    
    
    def _audio_transformation(self, is_train: bool=True):

        return torch.nn.Sequential(
                torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
                torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
                torchaudio.transforms.TimeMasking(time_mask_param=100)
            ) if is_train else torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)
    

    def _pitch_shift_aug(self, waveform, sample_rate):
        """
        Take in a waveform and return a waveform
        """
        # Choose a random pitch shift factor between -4 and +4 semitones
        pitch_shift_factor = random.uniform(-4.0, 4.0)

        # Apply the pitch shift effect
        pitch_transform = torchaudio.transforms.PitchShift(sample_rate=sample_rate, n_fft=1024, n_steps=pitch_shift_factor)
        pitch_shifted_waveform = pitch_transform(waveform)
        return pitch_shifted_waveform
    
    def _add_noise_aug(self, waveform):
        # Add random noise to the soundwave tensor array
        num_samples = waveform.shape[-1]
        num_noise_samples = int(num_samples * 0.1) # add 10% of noise samples
        noise_indices = torch.randint(low=0, high=num_samples, size=(num_noise_samples,))
        noise_values = torch.randn(size=(num_noise_samples,)) # generate random noise values
        waveform[0, noise_indices] += noise_values # add noise to the first channel of the waveform tensor array
        return waveform
            

    def data_processing(self, data, data_type='train'):

        """
        Process the audio data to retrieve the spectrograms that will be used for the training
        """

        text_transform = TextTransform()
        spectrograms = []
        input_lengths = []
        audio_path_list = []

        audio_transforms = self._audio_transformation(is_train=True) if data_type == 'train' else self._audio_transformation(is_train=False)

        if data_type != 'test':  
            labels = []
            label_lengths = []

            for audio_path, waveform, utterance in data:
                
                spec = audio_transforms(waveform).squeeze(0).transpose(0, 1)
                spectrograms.append(spec)
                label = torch.Tensor(text_transform.text_to_int(utterance))
                labels.append(label)
                input_lengths.append(spec.shape[0]//2)
                label_lengths.append(len(label))
                if (random.uniform(-1.0, 1.0) == 0):
                    augmented_spec = audio_transforms(self._pitch_shift_aug(waveform, 16)).squeeze(0).transpose(0, 1)
                    spectrograms.append(augmented_spec)
                    labels.append(label)
                    input_lengths.append(spec.shape[0]//2)
                    label_lengths.append(len(label))
                    print('change pitch')
                if (random.uniform(-1.0, 1.0) == 0):
                    augmented_spec = audio_transforms(self._add_noise_aug(waveform)).squeeze(0).transpose(0, 1)
                    spectrograms.append(augmented_spec)
                    labels.append(label)
                    input_lengths.append(spec.shape[0]//2)
                    label_lengths.append(len(label))
                    print('Add noise')

            spectrograms = torch.nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
            labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True)
            return audio_path, spectrograms, labels, input_lengths, label_lengths

        else:
            for audio_path, waveform in data:

                spec = audio_transforms(waveform).squeeze(0).transpose(0, 1)
                spectrograms.append(spec)
                input_lengths.append(spec.shape[0]//2)
                audio_path_list.append(audio_path)

            spectrograms = torch.nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
            return audio_path_list, spectrograms, input_lengths

In [153]:
# Decodes the logits into characters to form the final transciption using the greedy decoding approach

import torch
from typing import List
from torchaudio.models.decoder import ctc_decoder

class GreedyDecoder:

    """
    Decodes the logits into characters to form the final transciption using the greedy decoding approach
    """

    def __init__(self) -> None:
        pass


    def decode(
            self, 
            output: torch.Tensor, 
            labels: torch.Tensor=None, 
            label_lengths: List[int]=None, 
            collapse_repeated: bool=True, 
            is_test: bool=False
        ):
        
        """
        Main method to call for the decoding of the text from the predicted logits
        """
        
        text_transform = TextTransform()
        arg_maxes = torch.argmax(output, dim=2)
        decodes = []

        # refer to char_map_str in the TextTransform class -> only have index from 0 to 26, hence 27 represents the case where the character is decoded as blank (NOT <SPACE>)
        decoded_blank_idx = text_transform.get_char_len()

        if not is_test:
            targets = []

        for i, args in enumerate(arg_maxes):
            decode = []

            if not is_test:
                targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))

            for j, char_idx in enumerate(args):
                if char_idx != decoded_blank_idx:
                    if collapse_repeated and j != 0 and char_idx == args[j-1]:
                        continue
                    decode.append(char_idx.item())
            decodes.append(text_transform.int_to_text(decode))

        return decodes, targets if not is_test else decodes
    

class BeamSearchDecoder:

    def __init__(self) -> None:
        pass

    def decode(self, 
            output: torch.Tensor, 
            labels: torch.Tensor=None, 
            label_lengths: List[int]=None, 
            collapse_repeated: bool=True, 
            is_test: bool=False,
            beam_size=1500, 
            lm_weight=3.23, 
            word_score=-0.26, 
        ):
        text_transform = TextTransform()
        beam_search_decoder = ctc_decoder(
                                lexicon=None,
                                tokens=text_transform.get_char_list(),
                                nbest=3,
                                beam_size=beam_size,
                                lm=None,
                                lm_weight=lm_weight,
                                lm_dict=None,
                                word_score=word_score,
                                blank_token=' ',
                                sil_token=' ',
                            )
        arg_maxes = torch.argmax(output, dim=2)
        if not is_test:
            targets = []
        decodes = []
        result = beam_search_decoder(output.to(torch.float32))
        
        for i, args in enumerate(arg_maxes):
            statement = " ".join(result[i][0].words).strip()
            decodes.append(statement.upper())
            if not is_test:
                targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        
        return decodes, targets if not is_test else decodes




        
    

In [154]:

# The helper class for the training loop to do model training

import torch
import torch.nn.functional as F
from jiwer import wer, cer
import Levenshtein as leven

class IterMeter(object):

    """
    Keeps track of the total iterations during the training and validation loop
    """
    
    def __init__(self) -> None:
        self.val = 0


    def step(self):
        self.val += 1


    def get(self):
        return self.val
    

class TrainingLoop:

    """
    The main class to set up the training loop to train the model
    """

    def __init__(self) -> None:
        pass
    

    def train(self, model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter) -> None:

        """
        Training Loop
        """
        text_transform = TextTransform()
        beam_search = BeamSearchDecoder()

        model.train()
        data_len = len(train_loader.dataset)
        all_train_decoded = []
        all_train_actual = []
        
        for batch_idx, _data in enumerate(train_loader):
            audio_path, spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()

            optimizer.step()
            iter_meter.step()
            if batch_idx > (data_len - 5):
                model.eval()
                with torch.no_grad():
                    decoded = beam_search.decode(spectrograms)
                    for j in range(0, len(decoded)):
                        actual = text_transform.int_to_text(labels.cpu().numpy()[j][:label_lengths[j]].tolist())
                        all_train_decoded.append(decoded[j])
                        all_train_actual.append(actual)
                        train_levenshtein += leven.distance(decoded[j], actual)
                        len_levenshtein += label_lengths[j]
            
            if batch_idx % 100 == 0 or batch_idx == data_len:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(spectrograms), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))


    def dev(self, model, device, dev_loader, criterion, scheduler, epoch, iter_meter) -> None:

        """
        Validation Loop
        """
        
        print('\nevaluating...')
        model.eval()
        val_loss = 0
        test_cer, test_wer = [], []
        beam_search = BeamSearchDecoder()
        
        with torch.no_grad():
            for i, _data in enumerate(dev_loader):
                audio_path, spectrograms, labels, input_lengths, label_lengths = _data 
                spectrograms, labels = spectrograms.to(device), labels.to(device)

                output = model(spectrograms)  # (batch, time, n_class)
                output = F.log_softmax(output, dim=2)
                output = output.transpose(0, 1) # (time, batch, n_class)

                loss = criterion(output, labels, input_lengths, label_lengths)
                val_loss += loss.item() / len(dev_loader)

                # decoded_preds, decoded_targets = model.beam_search_with_lm(output.transpose(0, 1), labels=labels, label_lengths=label_lengths, is_test=False)
                decoded_preds, decoded_targets = beam_search.decode(output.transpose(0, 1), labels=labels, label_lengths=label_lengths, is_test=False)

                print(decoded_preds)
                print(decoded_targets)
                
                for j in range(len(decoded_preds)):
                    test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                    test_wer.append(wer(decoded_targets[j], decoded_preds[j]))

        avg_cer = sum(test_cer)/len(test_cer)
        avg_wer = sum(test_wer)/len(test_wer)
        
        scheduler.step(val_loss)

        print('Dev set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(val_loss, avg_cer, avg_wer))

In [155]:

# building the model with adaption of deepspeech2 -> https://arxiv.org/abs/1512.02595

# code adapted from https://towardsdatascience.com/customer-case-study-building-an-end-to-end-speech-recognition-model-in-pytorch-with-assemblyai-473030e47c7c


import torch


class CNNLayerNorm(torch.nn.Module):
    
    """
    Layer normalization built for CNNs input
    """
    
    def __init__(self, n_feats: int) -> None:
        super(CNNLayerNorm, self).__init__()

        self.layer_norm = torch.nn.LayerNorm(n_feats)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Input x of dimension -> (batch, channel, feature, time)
        """
        
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)

        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(torch.nn.Module):

    """
    Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf except with layer norm instead of batch norm
    """
    
    def __init__(self, in_channels: int, out_channels: int, kernel: int, stride: int, dropout: float, n_feats: int) -> None:
        super(ResidualCNN, self).__init__()

        self.cnn1 = torch.nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = torch.nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = torch.nn.Dropout(dropout)
        self.dropout2 = torch.nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)
        self.gelu = torch.nn.GELU()


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        """
        Model building for the Residual CNN layers
        
        Input x of dimension -> (batch, channel, feature, time)
        """

        residual = x
        x = self.layer_norm1(x)
        x = self.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = self.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual

        return x # (batch, channel, feature, time)


class BidirectionalGRU(torch.nn.Module):

    """
    The Bidirectional GRU composite code block which will be used in the main SpeechRecognitionModel class
    """
    
    def __init__(self, rnn_dim: int, hidden_size: int, dropout: int, batch_first: int) -> None:
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = torch.nn.GRU(
            input_size=rnn_dim, 
            hidden_size=hidden_size,
            num_layers=1, 
            batch_first=batch_first, 
            bidirectional=True
        )
        self.layer_norm = torch.nn.LayerNorm(rnn_dim)
        self.dropout = torch.nn.Dropout(dropout)
        self.gelu = torch.nn.GELU()


    def forward(self, x: torch.Tensor) -> torch.Tensor:

        """
        Transformation of the layers in the Bidirectional GRU block
        """

        x = self.layer_norm(x)
        x = self.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)

        return x


class SpeechRecognitionModel(torch.nn.Module):

    """
    The main ASR Model that the main code will interact with
    """
    
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1) -> None:
        super(SpeechRecognitionModel, self).__init__()
        
        n_feats = n_feats//2
        self.cnn = torch.nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = torch.nn.Sequential(*[
            ResidualCNN(
                in_channels=32, 
                out_channels=32, 
                kernel=3, 
                stride=1, 
                dropout=dropout, 
                n_feats=n_feats
            ) for _ in range(n_cnn_layers)
        ])
        self.fully_connected = torch.nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = torch.nn.Sequential(*[
            BidirectionalGRU(
                rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                hidden_size=rnn_dim, 
                dropout=dropout, 
                batch_first=i==0
            ) for i in range(n_rnn_layers)
        ])
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            torch.nn.GELU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(rnn_dim, n_class)
        )


    def forward(self, x: torch.Tensor) -> torch.Tensor:

        """
        Transformation of the layers in the ASR model block
        """

        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        
        return x

In [156]:
# main function
def main(hparams, train_dataset, dev_dataset, saved_model_path) -> None:

    """
    The main method to call to do model training
    """ 

    use_cuda = torch.cuda.is_available()
    torch.manual_seed(SEED)
    
    data_processor = DataProcessor()
    iter_meter = IterMeter()
    text_transform = TextTransform()
    trainer = TrainingLoop()
    
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=hparams['batch_size'],
        shuffle=True,
        collate_fn=lambda x: data_processor.data_processing(x, 'train'),
        **kwargs
    )
    
    dev_loader = torch.utils.data.DataLoader(
        dataset=dev_dataset,
        batch_size=hparams['batch_size'],
        shuffle=False,
        collate_fn=lambda x: data_processor.data_processing(x, 'dev'),
        **kwargs
    )

    model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], 
        hparams['n_rnn_layers'], 
        hparams['rnn_dim'],
        hparams['n_class'], 
        hparams['n_feats'], 
        hparams['stride'], 
        hparams['dropout']
    ).to(device)

    print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

    optimizer = torch.optim.AdamW(model.parameters(), hparams['learning_rate'])
    loss_fn = torch.nn.CTCLoss(blank=text_transform.get_char_len()).to(device)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', patience=3, verbose=True, factor=0.05)
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], steps_per_epoch=int(len(train_loader)), epochs=hparams['epochs'], anneal_strategy='linear')
    
    for epoch in range(1, hparams['epochs'] + 1):
        trainer.train(model, device, train_loader, loss_fn, optimizer, scheduler, epoch, iter_meter)
        trainer.dev(model, device, dev_loader, loss_fn, scheduler, epoch, iter_meter)
        
    # save the trained model
    torch.save(model.state_dict(), saved_model_path)

In [157]:
from time import time

# setting the random seed for reproducibility
SEED = 2022

# calling the model
hparams = {
            "n_cnn_layers": 7,
            "n_rnn_layers": 5,
            "rnn_dim": 512,
            "n_class": 28, # 26 alphabets in caps + <SPACE> + blanks
            "n_feats": 128,
            "stride": 2,
            "dropout": 0.2,
            "learning_rate": 5e-4,
            "batch_size": 16,
            "epochs": 50
      }
start_time = time()

# start training the model
main(
    hparams=hparams, 
    train_dataset=dataset_train, 
    dev_dataset=dataset_dev, 
    saved_model_path=SAVED_MODEL_PATH
)

end_time = time()

print(f"Time taken for training: {(end_time-start_time)/(60*60)} hrs")

Num Model Parameters 23704860
data length:  80


c:\Users\cheng yang\Anaconda3\lib\site-packages\torchaudio\functional\functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


1
Train Epoch: 1 [0/80 (0%)]	Loss: 7.677923
2
3
4
5

evaluating...
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['THE BUS ARRIVES EVERY TWO HOURS AT THIS STATION', 'THE PROJECT WILL TAKE TWO MONTHS TO COMPLETE', 'SHE BOUGHT THREE NEW DRESSES FOR THE WEDDING', 'THEY ONLY HAD SEVEN MINUTES TO GET READY BEFORE LEAVING', 'THERE IS ONLY ONE RULE AND THAT IS NO CHEATING', 'HE CAUGHT EIGHT FISH ON THEIR FISHING TRIP', 'THERE CAN ONLY BE ONE WINNER FOR THE CONTEST', 'THEY STAYED IN THE HOTEL ROOM FOR SEVEN NIGHTS', 'THEY HAD TO CLIMB FOUR FLIGHTS OF STAIRS TO REACH THE TOP', 'SHE DONATED NINE BAGS OF CLOTHES TO CHARITY', 'SHE RECEIVED NINE BIRTHDAY CARDS IN THE MAIL', 'SHE SPENT SIX WEEKS VOLUNTEERING AT AN ANIMAL SHELTER', 'THE SEVEN SAMURAI IS A FAMOUS JAPANESE FILM', 'THEY FOUND SEVEN DIFFERENT SPECIES OF FLOWERS IN THE GARDEN', 'SHE WON SIX AWARDS FOR HER ARTWORK', 'THE BUS ARRIVES EVERY TWO HOURS AT THIS STATION']
['', '', '', '']
['SHE IS ONE OF THE MOST GENEROUS PEOP